In [2]:
import os

import pandas as pd
import numpy as np
from scipy import stats
from scipy import io as sio
from scipy.optimize import least_squares
import matplotlib.pyplot as plt
from matplotlib import patches

In [3]:
def wrp(x):
    bigger90=x>np.pi/2
    smaller90=x<-np.pi/2
    x[bigger90]=x[bigger90]-np.pi
    x[smaller90]=x[smaller90]+np.pi
    return x
def wrpdeg(x):
    bigger90=x>90
    smaller90=x<-90
    x[bigger90]=x[bigger90]-180
    x[smaller90]=x[smaller90]+180
    return x

In [8]:
df=pd.read_csv('sub208_rt_rs_deidentify.csv')
# final subjects
print(df.shape[0]/120)

208.0


In [9]:
# precision ACROSS subjects
rdf=df[df['debadtrial']!=True]
precisiondf=rdf[['id','abserror','deabserror']].groupby('id').mean()
precisiondf

,abserror,deabserror
id,,
1,25.362832,9.274336
2,25.868421,12.631579
3,10.273504,9.529915
4,27.517241,9.767241
5,28.252174,11.347826
...,...,...
204,13.271930,5.640351
205,31.901786,26.892857
206,7.965217,5.347826


In [7]:
print("1-item trial")
print(np.mean(np.array(precisiondf['deabserror']))," ± ",np.std(np.array(precisiondf['deabserror'])))
print("3-item trial")
print(np.mean(np.array(precisiondf['abserror']))," ± ",np.std(np.array(precisiondf['abserror'])))

1-item trial
10.207627751401889  ±  5.586757097212392
3-item trial
20.541134704046506  ±  8.547422458215115


In [8]:
deallmask=df['debadtrial']==True
deallfmask=df['debadtrial']!=True
df.loc[deallmask, 'deerror'] = np.nan
meanbias=np.nanmean(np.array(df['deerror']))
print(meanbias)
# print(df.loc[deallfmask, 'deerror'][0:20])
df.loc[deallfmask, 'deerror']=wrpdeg(df.loc[deallfmask, 'deerror']-meanbias)
# print(df.loc[deallfmask, 'deerror'][0:20])
hdf=df[df['trialconflict']==2]
ldf=df[df['trialconflict']==1]
hdftc=hdf[hdf['testc']==1]
ldftc=ldf[ldf['testc']==1]

-0.8527801179443977


In [9]:
#### Remove bad trial
h_badde_mask=hdf['debadtrial']==True
hdf.loc[h_badde_mask, 'deerror'] = np.nan
l_badde_mask=ldf['debadtrial']==True
ldf.loc[l_badde_mask, 'deerror'] = np.nan

hpmidirec=np.array(hdf['pmiori'])
himidirec=np.array(hdf['imiori'])
hcdirec=np.array(hdf['cori'])
hdedirec=np.array(hdf['deori'])
hdeerror=np.array(hdf['deerror'])

lpmidirec=np.array(ldf['pmiori'])
limidirec=np.array(ldf['imiori'])
lcdirec=np.array(ldf['cori'])
ldedirec=np.array(ldf['deori'])
ldeerror=np.array(ldf['deerror'])


h_ind = ~np.isnan(hdeerror)
h_pmi = np.deg2rad(hpmidirec[h_ind])
h_imi = np.deg2rad(himidirec[h_ind])
h_c = np.deg2rad(hcdirec[h_ind])
h_de = np.deg2rad(hdedirec[h_ind])

l_ind = ~np.isnan(ldeerror)
l_pmi = np.deg2rad(lpmidirec[l_ind])
l_imi = np.deg2rad(limidirec[l_ind])
l_c = np.deg2rad(lcdirec[l_ind])
l_de = np.deg2rad(ldedirec[l_ind])

h_deerror = np.deg2rad(hdeerror[h_ind])
l_deerror = np.deg2rad(ldeerror[l_ind])

highdf=pd.DataFrame(data={'pmi':h_pmi,'imi':h_imi,'c':h_c,'de':h_de,'de_error':h_deerror})
lowdf=pd.DataFrame(data={'pmi':l_pmi,'imi':l_imi,'c':l_c,'de':l_de,'de_error':l_deerror})

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [10]:
def dog(x, a, w):
    c = np.sqrt(2) / np.exp(-0.5)
    return x * a * w * c * np.exp(-(w * x) ** 2)


def fit_dog(y, x):

    def _solver(params):
        a, w = params
        return y - dog(x, a, w)

    min_a = -np.pi/2
    max_a = np.pi/2

    min_w = 0.8
    max_w = 8.0

    min_cost = np.inf
    for _ in range(200):
        params_0 = [np.random.rand() * (max_a - min_a) + min_a,
                    np.random.rand() * (max_w - min_w) + min_w]
        try:
            result = least_squares(_solver, params_0,
                                   bounds=([min_a, min_w],
                                           [max_a, max_w]))
        except ValueError:
            continue
        if result['cost'] < min_cost:
            best_params, min_cost = result['x'], result['cost']
    try:
        return best_params[0], best_params[1], min_cost
    except UnboundLocalError:
        return np.nan, np.nan, min_cost

In [11]:
def indirect(a1, w1,a2,w2,imi,c,de):
    d_c_imi=wrp(imi-c)
    bias_c=dog(d_c_imi, a1, w1)
    c_prime=wrp(c+bias_c)
    d_de_cprime=wrp(c_prime-de)
    bias_de = dog(d_de_cprime, a2, w2)

    return bias_de

def fit_indirect(y, imi,c,de):

    def _solver(params):
        a1, w1,a2,w2 = params
        return y - indirect(a1, w1,a2,w2,imi,c,de)

    min_a = -np.pi/2
    max_a = np.pi/2

    min_w = 0.8
    max_w = 8.0

    min_cost = np.inf
    for _ in range(200):
        params_0 = [np.random.rand() * (max_a - min_a) + min_a,
                    np.random.rand() * (max_w - min_w) + min_w,
                    np.random.rand() * (max_a - min_a) + min_a,
                    np.random.rand() * (max_w - min_w) + min_w]
        try:
            result = least_squares(_solver, params_0,
                                   bounds=([min_a, min_w,min_a, min_w],[max_a, max_w,max_a, max_w]))
        except ValueError:
            continue
        if result['cost'] < min_cost:
            best_params, min_cost = result['x'], result['cost']
    try:
        return best_params, min_cost
    except UnboundLocalError:
        return np.nan, np.nan
    
def direct(a1, w1,imi,de):
    d_de_imi=wrp(imi-de)
    bias_de = dog(d_de_imi, a1, w1)

    return bias_de

def fit_direct(y, imi,de):

    def _solver(params):
        a1, w1 = params
        return y - direct(a1, w1,imi,de)

    min_a = -np.pi/2
    max_a = np.pi/2

    min_w = 0.8
    max_w = 8.0

    min_cost = np.inf
    for _ in range(200):
        params_0 = [np.random.rand() * (max_a - min_a) + min_a,
                    np.random.rand() * (max_w - min_w) + min_w]
        try:
            result = least_squares(_solver, params_0,
                                   bounds=([min_a, min_w],[max_a, max_w]))
        except ValueError:
            continue
        if result['cost'] < min_cost:
            best_params, min_cost = result['x'], result['cost']
    try:
        return best_params, min_cost
    except UnboundLocalError:
        return np.nan, np.nan
    
def fit_zero(y):

    def _solver(params):
        a1, w1 = params
        return y - 0

    min_a = -np.pi/2
    max_a = np.pi/2

    min_w = 0.8
    max_w = 8.0

    min_cost = np.inf
    for _ in range(200):
        params_0 = [np.random.rand() * (max_a - min_a) + min_a,
                    np.random.rand() * (max_w - min_w) + min_w]
        try:
            result = least_squares(_solver, params_0,
                                   bounds=([min_a, min_w],[max_a, max_w]))
        except ValueError:
            continue
        if result['cost'] < min_cost:
            best_params, min_cost = result['x'], result['cost']
    try:
        return best_params, min_cost
    except UnboundLocalError:
        return np.nan, np.nan

In [13]:
# AIC=n*log(SSR/n)+2k
# AICc=AIC+2k(k+1)/(n-k-1)

h_indirect_para,h_indirect_SSR=fit_indirect(h_deerror, h_imi,h_c,h_de)
h_indirect_SSR=h_indirect_SSR*2
n=len(h_deerror)
n_para=4
print('HIGH indirect model AICc:')
print(n*np.log(h_indirect_SSR/n)+2*n_para+2*n_para*(n_para+1)/(n-n_para-1))

h_direct_para,h_direct_SSR=fit_direct(h_deerror, h_imi,h_de)
h_direct_SSR=h_direct_SSR*2
n=len(h_deerror)
n_para=2
print('HIGH direct model AICc:')
print(n*np.log(h_direct_SSR/n)+2*n_para+2*n_para*(n_para+1)/(n-n_para-1))

h_zero_para,h_zero_SSR=fit_zero(h_deerror)
h_zero_SSR=h_zero_SSR*2
n=len(h_deerror)
n_para=0
print('HIGH null model AICc:')
print(n*np.log(h_zero_SSR/n)+2*n_para+2*n_para*(n_para+1)/(n-n_para-1))

HIGH indirect model AICc:
-32944.264850113104
HIGH direct model AICc:
-32913.818855246784
HIGH null model AICc:
-32915.51398204353


In [14]:
print(32944.264850113104-32915.51398204353)
print(32915.51398204353-32913.818855246784)

28.750868069575517
1.6951267967451713


In [15]:
h_indirect_para

array([0.2773078 , 2.55709676, 0.02319403, 1.51296336])

In [15]:
# print(31809.308371585277-31774.612292395625)
# print(31774.612292395625-31770.826471270182)

34.696079189652664
3.785821125442453


In [16]:
# AIC=n*log(SSR/n)+2k
# AICc=AIC+2k(k+1)/(n-k-1)

l_indirect_para,l_indirect_SSR=fit_indirect(l_deerror, l_imi,l_c,l_de)
l_indirect_SSR=l_indirect_SSR*2
n=len(l_deerror)
n_para=4
print('LOW indirect model AICc:')
print(n*np.log(l_indirect_SSR/n)+2*n_para+2*n_para*(n_para+1)/(n-n_para-1))

l_direct_para,l_direct_SSR=fit_direct(l_deerror, l_imi,l_de)
l_direct_SSR=l_direct_SSR*2
n=len(l_deerror)
n_para=2
print('LOW direct model AICc:')
print(n*np.log(l_direct_SSR/n)+2*n_para+2*n_para*(n_para+1)/(n-n_para-1))

l_zero_para,l_zero_SSR=fit_zero(l_deerror)
l_zero_SSR=l_zero_SSR*2
n=len(l_deerror)
n_para=0
print('LOW zero model AICc:')
print(n*np.log(l_zero_SSR/n)+2*n_para+2*n_para*(n_para+1)/(n-n_para-1))

LOW indirect model AICc:
-31811.325838695928
LOW direct model AICc:
-31772.827485468948
LOW zero model AICc:
-31776.612630376192


In [17]:
print(31811.325838695928-31776.612630376192)
print(31776.612630376192-31772.827485468948)

34.71320831973571
3.785144907244103


In [18]:
l_indirect_para

array([0.13122957, 1.90767793, 0.02326077, 1.18985393])